In [1]:
!pip install geopy
!pip install unidecode

You should consider upgrading via the 'C:\Users\HP ProBook 640\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import os
from geopy.geocoders import Nominatim
from geopy import distance
import unidecode
from Geocoding.Geocoding import google_maps, geo_py
from Weather.Weather import weather_py

In [ ]:
path_base_source = '..\\..\\Bases\\Clima\\BasesClimaOrigem\\'
path_base_final = '..\\..\\Bases\\Clima\\'
path_municipio_base ='..\\..\\Bases\\Municipios\\'

file_municipios = 'Municipios.csv'

In [3]:

file_path = '{}{}'.format(path_municipio_base,file_municipios)
df_municipios = pd.read_csv(file_path, sep=";", decimal=',', encoding='utf-8')

In [4]:
# Transforma a coluna 'cidade' em uma lista, removendo espaços e adicionando '+'
lista_cidades = [cidade.replace(' ', '+') for cidade in df_municipios['Cidades'].tolist()]
lista_cidades_csv = ['PORTUGAL,+' + cidade.upper() +'.CSV' for cidade in lista_cidades]

In [5]:
csv_files = [x.upper() for x in os.listdir(path_base_source) if '.csv' in x]

In [6]:
os.chdir(path_base_final)
os.getcwd()

'G:\\Users\\johns\\Desktop\\TCC\\MPAlugueisLisboa\\Bases\\ClimaPortugal'

In [7]:
# processa Arquivo de Clima
diferenca = set(lista_cidades_csv) - set(csv_files)

if diferenca is not None:
    weather = weather_py()
    for location in diferenca:
        weather.process_wheather(
            location_list = [location],
            frequency=24, 
            start_date="2021-01-01", 
            end_date="2021-12-30",
            retries=10, 
            fileDone="..\\Temporaria\\done.csv",
            resetDoneFile=False
        )


In [8]:
# Agrega bases de clima
csv_files = [x.upper() for x in os.listdir(path_base_source) if '.csv' in x]
df_clima = pd.concat([pd.read_csv(path_base_source + f ,  sep=',') for f in csv_files])
df_clima.rename(columns={'location':'Cidades'},inplace=True)

In [10]:
df_clima['Cidades'] = df_clima['Cidades'].str.replace("+"," ")
df_clima['Cidades'] = df_clima['Cidades'].str.replace("Brasil, ","")
df_clima['Cidades'] = df_clima['Cidades'].str.replace("Portugal, ","")
df_clima['Cidades'] = df_clima['Cidades'].apply(lambda x: unidecode.unidecode(x)) 

C:\Users\johns\AppData\Local\Temp\ipykernel_21216\3732271281.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_clima['Cidades'] = df_clima['Cidades'].str.replace("+"," ")


In [11]:
df_clima['Date'] = pd.to_datetime(df_clima['date_time'])
df_clima['YEAR'] = df_clima['Date'].dt.strftime('%Y')
df_clima['MONTH'] = df_clima['Date'].dt.strftime('%m')
df_clima['YEAR_MONTH'] = df_clima['YEAR']+df_clima['MONTH']
df_clima['QUARTER'] = pd.PeriodIndex(df_clima['Date'], freq= 'Q').quarter

In [12]:
df_clima_ano_mes = (
    df_clima
    #.query('order_approved_at < "2018-01-01"')
    .groupby(['Cidades','YEAR_MONTH','YEAR','MONTH','QUARTER'])
    .agg(Temp_Median_Max = ('maxtempC', 'median'),
         Temp_Median_Min = ('mintempC', 'median'),
         Temp_Mean_Max = ('maxtempC', 'mean'),
         Temp_Mean_Min = ('mintempC', 'mean'))
    .reset_index().round(2) 
)


In [ ]:
# Exporta Base Mensal
file_name = "{}{}{}".format(path_base_final,"BaseClimaMensal",".csv")
df_clima_ano_mes.to_csv(file_name, sep=';', quotechar='"', decimal=',')


In [14]:
# Cria base de clima pivoteada
df_clima_ano_mes_pivot = (
    pd
    .pivot_table(df_clima_ano_mes, values='Temp_Median_Max', index='Cidades', columns='YEAR_MONTH', aggfunc=np.sum)
    .reset_index()
)


In [15]:
file_name = "{}{}{}".format(path_base_final,"BaseClimaMensalPivot",".csv")
df_clima_ano_mes_pivot.to_csv(file_name, sep=';', quotechar='"', decimal=',')

In [16]:
# Cria Ranking Geral de todas regioes
list_var = list(df_clima_ano_mes_pivot.columns)[1:]

In [17]:
for var in list_var:
    hot = df_clima_ano_mes_pivot[var].describe()['75%']
    neutral = df_clima_ano_mes_pivot[var].describe()['50%']
    frost = df_clima_ano_mes_pivot[var].describe()['50%']
    
    new_col = 'RESULT_' + var
    new_col = new_col.replace(" ", "_")
    df_clima_ano_mes_pivot[new_col] = 0
    df_clima_ano_mes_pivot.loc[df_clima_ano_mes_pivot[var] >= hot, new_col] = 3
    df_clima_ano_mes_pivot.loc[((df_clima_ano_mes_pivot[var] >=  neutral) & (df_clima_ano_mes_pivot[var] < hot)),new_col]= 2
    df_clima_ano_mes_pivot.loc[df_clima_ano_mes_pivot[var] < frost, new_col] = 1
    
    lst_columns = [columns for columns in df_clima_ano_mes_pivot if 'RESULT_' in columns]
    
    df_clima_ano_mes_pivot['Clima_Ranking'] = df_clima_ano_mes_pivot[lst_columns].sum(axis=1)
    
    df_clima_ranking = (
        df_clima_ano_mes_pivot
        .filter(['Cidades','Clima_Ranking'])
        .sort_values('Clima_Ranking', ascending = False)
    )

In [ ]:
file_name = "{}{}{}".format(path_base_final,'RankingClima',".csv")
df_clima_ranking.to_csv(file_name, sep=';', quotechar='"')


In [20]:
# adiciona à base principal

file_path = '{}{}'.format(path_municipio_base,"Municipios_geo.csv")
df_municipios = pd.read_csv(file_path, sep=";") 
df_municipios['Cidades'] = df_municipios['Cidades'].apply(lambda x: unidecode.unidecode(x))

In [21]:
file_path = '{}{}'.format(path_base_final,"RankingClima.csv")
df_clima_ranking = pd.read_csv(file_path, sep=";")
df_clima_ranking['Cidades'] = df_clima_ranking['Cidades'].apply(lambda x: unidecode.unidecode(x))

In [22]:
lst_c = list(df_municipios.columns)
lst_c.append('Clima_Ranking')
df_municipios = (
    df_municipios
    .filter(lst_c)
    .merge(df_clima_ranking,left_on='Cidades', right_on='Cidades', how='left')
)

In [23]:
# Exporta Base Minucipios
file_name = "{}{}{}".format(path_municipio_base,'Municipios_geo_clima',".csv")
df_municipios.to_csv(file_name, sep=';', quotechar='"', decimal=',')